# Forecasting stream gap analysis for future surveys

Codebase & instructions for the model of Hendel & Bovy 2020, based primarily on the **streampepper** formalism of Bovy, Erkal & Sanders 2017

The method is designed to be extensible to new streams and/or survey parameters.

To add new streams, one needs
1. The stream's orbit, age, & velocity dispersion
2. Star counts in an existing survey
3. An assumed stellar population (age, metallicity, and stellar mass function slope)
4. A choice of coordinate system & mock survey area
5. (optional) To include background stars, a Galaxia (Sharma et al. 2011) mock halo survey at the stream position

To add a new survey, one needs
1. Isochrone tables (preferably matched to the Galaxia grid, see isochrone_handling.py)
2. An error model, e.g. $\Delta_{r}(r)$, in at least two photometric bands.

In [1]:
%matplotlib widget
import os
import sys
import time
import subprocess
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

## Setting up stream models

First we create the streampepper dynamical models for each stream. This is done by make_streampepper_models file which has the necessary parameters and uses them to call the streammodel_util.py file. The latter generalizes the old pal5_util and gd1_util files. There will be a separate leading-tail and trailing-tail model for each. We also introduce the coordinate transformations here for convenience; each model has the Astropy frame as sdf.stream_coord for convenience.

Depending on the potential, orbit, and desired number of times that impacts can happen these models can take quite some time to generate so the script can be run with multiprocessing enabled, if desired. It returns pickles of the models for quick restarts after the inital calculation.

New streams can be easily added by reproducing the format seen for Pal5, Phoenix and GD-1.

In [ ]:
#python3 make_streampepper_models.py